In [7]:
from lib import utilities
import numpy as np
import copy
from scipy.linalg import expm
from qiskit.utils.mitigation.fitters import CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import  complete_meas_cal
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q-community',
                            group='ibmquantumawards', 
                            project='open-science-22')
backend_sim_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_real_jakarta = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ibmqfactory.load_account:WARNING:2022-04-13 01:49:33,557: Credentials are already in use. The existing account in the session will be replaced.


In [15]:
n_steps = 99
n_points = 10
time_range = np.linspace(0.001, np.pi, n_points)
time_calibration = np.pi
decomposition = "SSD" # or "HSD"

############### circuits building
print("building circuits")
qr_cal = QuantumRegister(3)
calib_circuits_qiskit, state_labels = complete_meas_cal(qubit_list=[0,1,2], qr=qr_cal, circlabel='mcal')

evo_qc_calibration, evo_qr_calibration = utilities.get_evolution_circuit(time_calibration, n_steps, method=decomposition)
calib_circuits_NIC = utilities.get_calibration_circuits(evo_qc_calibration, method="NIC") #ATTENZIONE!!!! NON CREA LO STATO GIUSTO (111) PER LA WHOLE SPACE
calib_circuits_CIC = utilities.get_calibration_circuits(evo_qc_calibration, method="CIC")

qcs_tomography_SSD_list = []
for i, time in enumerate(time_range):
    evo_qc, evo_qr = utilities.get_evolution_circuit(time, n_steps, method=decomposition)
    qcs_tomography_HSD = state_tomography_circuits(evo_qc, evo_qr)
    qcs_tomography_SSD_list.append(qcs_tomography_HSD)

############### execute all
print("execute all circuits")
shots = 32000
# calibration circuits. For each techniqe are runned 8 (2^n) calibration circuits.
job_NIC=execute(calib_circuits_NIC, backend = backend_sim_jakarta, shots=shots, initial_layout=[1,3,5])
job_CIC=execute(calib_circuits_CIC, backend = backend_sim_jakarta, shots=shots, initial_layout=[1,3,5])
job_qiskit=execute(calib_circuits_qiskit, backend = backend_sim_jakarta, shots=shots, initial_layout=[1,3,5])

# evolution cicuits. Are runned 27 circuits for each point.
jobs_evo = []
for i in range(n_points):
    job=execute(qcs_tomography_SSD_list[i], backend = backend_sim_jakarta, shots=shots, initial_layout=[1,3,5])
    jobs_evo.append(job)

############### mitigation
print("mitigation")
# compute the U_ideal unitary matrix of the circuit. Is used only for the CIC mitigation.
U_ideal = utilities.matrix_from_circuit(evo_qc_calibration)
# fast_tomography_calibration_MeasFitters returns a list of 27 CompleteMeasFitter objects, one for each tomo basis.
meas_fitters_NIC = utilities.fast_tomography_calibration_MeasFitters(job_NIC.result(), method="NIC")
meas_fitters_CIC = utilities.fast_tomography_calibration_MeasFitters(job_CIC.result(), method="CIC", U_ideal=U_ideal)
meas_fitters_qiskit = CompleteMeasFitter(job_qiskit.result(), state_labels=state_labels)
# in meas_fitters_NIC and meas_fitters_CIC there are all the calibration matrices.

target_state = (One^One^Zero).to_matrix()

raw_result_list = []
qiskit_result_list = []
NIC_result_list = []
CIC_result_list = []

fids_list = [[],[],[],[]]

for i, time in enumerate(time_range):

    raw_result = jobs_evo[i].result()
    qiskit_result = copy.deepcopy(raw_result)
    NIC_result = copy.deepcopy(raw_result)
    CIC_result = copy.deepcopy(raw_result)

    for j in range(27):
        old_counts=raw_result.get_counts(j)
        new_counts_NIC = meas_fitters_NIC[j].filter.apply(old_counts, method="least_squares")
        NIC_result.results[j].data.counts = new_counts_NIC
        
        new_counts_CIC = meas_fitters_CIC[j].filter.apply(old_counts, method="least_squares")
        CIC_result.results[j].data.counts = new_counts_CIC

        new_counts_qiskit = meas_fitters_CIC[j].filter.apply(old_counts, method="least_squares")
        qiskit_result.results[j].data.counts = new_counts_qiskit

    for j, result in enumerate([raw_result, qiskit_result, NIC_result, CIC_result]):
        fids_list[j].append(utilities.fidelity_count(result, qcs_tomography_SSD_list[j], target_state))
    
    print(i)
############### results


building circuits
attenzione al caso != HSD o SSD.  -get_evolution_circuit
invertire lo stato iniziale se la decomposizione è HSD.
invertire lo stato iniziale se la decomposizione è HSD.
attenzione al caso != HSD o SSD.  -get_evolution_circuit
attenzione al caso != HSD o SSD.  -get_evolution_circuit
attenzione al caso != HSD o SSD.  -get_evolution_circuit
attenzione al caso != HSD o SSD.  -get_evolution_circuit
attenzione al caso != HSD o SSD.  -get_evolution_circuit
attenzione al caso != HSD o SSD.  -get_evolution_circuit
attenzione al caso != HSD o SSD.  -get_evolution_circuit
attenzione al caso != HSD o SSD.  -get_evolution_circuit
attenzione al caso != HSD o SSD.  -get_evolution_circuit
attenzione al caso != HSD o SSD.  -get_evolution_circuit
execute all circuits
mitigation
0
1
2
3
4
5
6
7
8
9


In [17]:
from qiskit.opflow import Zero, One, I, X, Y, Z

def H_heis3():
    # Interactions (I is the identity matrix; X, Y, and Z are Pauli matricies; ^ is a tensor product)
    XXs = (I^X^X) + (X^X^I)
    YYs = (I^Y^Y) + (Y^Y^I)
    ZZs = (I^Z^Z) + (Z^Z^I)
    # Sum interactions
    H = XXs + YYs + ZZs
    # Return Hamiltonian
    return H

def U_heis3(t):
    # Compute XXX Hamiltonian for 3 spins in a line
    H = H_heis3()
    # Return the exponential of -i multipled by time t multipled by the 3 spin XXX Heisenberg Hamilonian 
    return (t * H).exp_i()

# Define initial state |110>
initial_state = One^One^Zero

def prob_state(initial_state, time_range):
    return [np.abs((~initial_state @ U_heis3(float(t)) @ initial_state).eval())**2 for t in time_range]

t_ideal = np.linspace(0,np.pi, 1000)
ideal_probs = {
    "110": prob_state(One^One^Zero, t_ideal),
    "101": prob_state(One^Zero^One, t_ideal),
    "011": prob_state(Zero^One^One, t_ideal)
}

labels = ["raw", "Qiskit", "NIC", "CIC"]

plt.figure(figsize=(13,10), dpi=1000)
for i, fids in enumerate(fids_list):
    plt.plot(time_range, fids, linestyle='', marker='*', label=labels[i])

for state in ["110", "101", "011"]:
    plt.plot(t_ideal, ideal_probs[state])

plt.legend()
plt.grid()
plt.xlabel("time")
plt.ylabel("fidelity")
plt.title(f"time evolution with {n_steps} trotter steps and with {decomposition} decomposition.")
plt.show()